In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt     
import seaborn as sns  
import missingno as msn   
import janitor
import re     
from dateutil import parser
# Install local package using "pip install -e . --config-setting editable_mode=compat"
import src  
from src.clean import load_data_and_dict       
from collections import Counter
import os
from dotenv import load_dotenv

In [3]:
# Load the west africa data 
load_dotenv() 

df = pd.read_excel(os.getenv('TBDATA_FILE_PATH'), sheet_name='COMBO_DATA')

df.head()

,Patient ID,Date of screening,Date of birth,Age in years,Sex,Was the child vaccinated at birth (BCG Vaccine)?,Is the child's HIV status known?,Is the child taking anti-retroviral therapy (ART) for HIV-infection?,Has the child ever had TB of the lung?,What YEAR was the child FIRST diagnosed with TB of the lung?,...,If yes Hilar/mediastinal calcification Lateral (choice=Right),Vertebral collapse Collapse,If yes Vertebral collapse Collapse (choice=Left),If yes Vertebral collapse Collapse (choice=Right),Conclusion,Xpert Ultra result? (FROM GENE XPERT REGISTER),Record the final Sputum or Gastric Aspirate A/B MGIT culture result.,Record the ZN result.,"If the MGIT culture was POSITIVE, record the DAYS to culture-positivity (as measured by the machine).","If the MGIT culture was POSITIVE, record the hours to culture-positivity (as measured by the machine)."
0,G21-001A,2021-11-04,2014-06-28,7.4,Male,Yes,Don't know,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,No radiologic abnormalities,Negative,Negative,Negative,NaN,NaN
1,G21-002E,2021-11-04,2012-10-30,9.0,Male,Yes,Don't know,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Abnormal - not necessarily compatible with TB,Negative,Negative,Negative,NaN,NaN
2,G21-003Z,2021-11-04,2009-03-13,12.6,Female,Yes,Don't know,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,No radiologic abnormalities,Negative,Negative,Negative,NaN,NaN
3,G21-004B,2021-11-04,2016-01-01,5.8,Male,Yes,Don't know,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Abnormal - Likely TB,Negative,Negative,Negative,NaN,NaN
4,G21-005P,2021-11-15,2013-01-01,8.9,Male,Don't Know,Don't know,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Abnormal - not necessarily compatible with TB,Negative,Negative,Negative,NaN,NaN


In [4]:
matching_columns = {
    "Patient ID": "patient_id",
    "Date of screening": "date_particip",
    "Age in years": "age",
    "Sex": "gender",
    "Cough": "cough",
    "Difficulty breathing": "difficulty_breathing",
    "Chest pain": "chest_pain",
    "Coughing up blood": "haemoptysis",
    "Wheezing": "wheeze",
    "Fever": "fever_chills",
    "Weight loss": "loss_weight",
    "Pallor": "pallor",
    "Jaundice": "jaundice",
    "Lymphadenopathy": "neck_lymph",
    "Hepatomegaly": "hepatomegaly",
    "Splenomegaly": "splenomegaly",
    "Rhonchi": "rhonchi",
    "Reduced air entry": "reduced_breath_sounds",
    "Crackles": "crepitation",
    "Record temperature (in Celsius)": "temperature",
    "Record weight (in kilograms)": "weight",
    "Record height (in centimeters)    (Recumbent length if <  2 years, standing height if older)": "height",
    "Mid-upper arm circumference (cm)": "mua_circumference",
    "Record respiratory rate (in breaths per minute)": "respiratory_rate",
    "HIV test result": "hiv_serology",
    "Xpert Ultra result?  (FROM GENE XPERT REGISTER)": "genexpert_result",
    "Record the final Sputum or Gastric Aspirate A/B MGIT culture result.": "mgit_result",
    "Record the ZN result.": "spec_result",
}
# Select and rename columns
df = df[list(matching_columns.keys())].rename(columns=matching_columns)

df.columns

Index(['patient_id', 'date_particip', 'age', 'gender', 'cough',
       'difficulty_breathing', 'chest_pain', 'haemoptysis', 'wheeze',
       'fever_chills', 'loss_weight', 'pallor', 'jaundice', 'neck_lymph',
       'hepatomegaly', 'splenomegaly', 'rhonchi', 'reduced_breath_sounds',
       'crepitation', 'temperature', 'weight', 'height', 'mua_circumference',
       'respiratory_rate', 'hiv_serology', 'genexpert_result', 'mgit_result',
       'spec_result'],
      dtype='object')

`There are 28 columns that don't match: productive, night_sweats, side_pain, malaise_fatigue, loss_appetite, vomiting, pretb_history, date_previous_tb, wasting, dyspnoea, lethargy, abd_distension_mass, other_clinical_sign, subscostal_rec, bronchial_breathing, other_abnorm, abnorm_site, xray_assessment, other_xray, specimen_type, sputum_type, other_mycobactera, histology_result, diagnosis, eptb_site, nottb_diagnosis, antitb_treatment, country`
